# Imports

In [1]:
import pandas as pd
import numpy as np

In [2]:
import nfl_data_py as nfl

# Play by Play Data

In [3]:
columns = None
years = [2023]
df_play = nfl.import_pbp_data(years, columns, downcast=True, cache=False, alt_path=None)

2023 done.
Downcasting floats.


## Column Exploration

In [4]:
s = "season"
for col in list(df_play.columns):
    if s in col:
        print(col)

season_type
season


In [5]:
df_play["season_type"].head()

0    REG
1    REG
2    REG
3    REG
4    REG
Name: season_type, dtype: object

# Who Throws Deep?

Finding average depth of target (adot) for quarterbacks.

Grab only the pass plays. Note, in pandas, you should use the `&` operator for element-wise logical AND operations.

In [6]:
df_pass_plays = df_play[(df_play["play_type"] == "pass") & (df_play["air_yards"].notna())]

## ADOT by QB

Group by 'passer_id' and 'passer', then aggregate air yards by count and mean.

In [7]:
grouped = df_pass_plays.groupby(['passer_id', 'passer']).agg({"air_yards": ["count", "mean"]})

Creating a refined data frame where we only keep players who have attempted over 100 passes. We sort, descending, by adot.

In [8]:
grouped.columns = list(map("_".join, grouped.columns.values))
sort_crit = "air_yards_count > 100"
print(grouped.query(sort_crit).sort_values(by="air_yards_mean", ascending=[False]).to_string())

                                air_yards_count  air_yards_mean
passer_id  passer                                              
00-0039152 W.Levis                          150       10.193334
00-0029701 R.Tannehill                      157        9.375796
00-0033537 D.Watson                         170        9.370588
00-0035228 K.Murray                         107        9.308412
00-0039163 C.Stroud                         385        9.241558
00-0036264 J.Love                           370        8.967567
00-0036389 J.Hurts                          358        8.575419
00-0034855 B.Mayfield                       380        8.497369
00-0034857 J.Allen                          432        8.284722
00-0033077 D.Prescott                       370        8.270270
00-0031280 D.Carr                           372        8.223118
00-0033275 P.Walker                         110        8.200000
00-0031345 J.Garoppolo                      167        8.119761
00-0037834 B.Purdy                      

## ADOT For Receivers

In [9]:
receivers = df_pass_plays.groupby(['receiver_id', 'receiver']).agg({"air_yards": ["count", "mean"]})

In [10]:
receivers.columns = list(map("_".join, receivers.columns.values))
sort_crit = "air_yards_count > 50"
print(receivers.query(sort_crit).sort_values(by="air_yards_mean", ascending=[False]).to_string())

                            air_yards_count  air_yards_mean
receiver_id receiver                                       
00-0036261  B.Aiyuk                      65       15.107693
00-0037545  R.Shaheed                    55       14.781818
00-0030564  D.Hopkins                    83       14.674699
00-0031408  M.Evans                      91       14.274725
00-0038977  N.Dell                       75       14.266666
00-0031544  A.Cooper                     86       14.139535
00-0036196  G.Davis                      67       13.880597
00-0037239  C.Olave                     103       13.757281
00-0035640  D.Metcalf                    80       13.550000
00-0037247  G.Pickens                    73       12.945206
00-0034837  C.Ridley                     76       12.671053
00-0036912  D.Smith                      76       12.460526
00-0035676  A.Brown                     105       12.447619
00-0038994  J.Addison                    82       11.975610
00-0035662  M.Brown                     

In [11]:
player_name = "D.Adams"
receivers.query(f"receiver == '{player_name}'")

,,air_yards_count,air_yards_mean
receiver_id,receiver,,
00-0031381,D.Adams,118,11.788136
